# Pandas 怎么新增数据列?
#### 在进行数据分析时,经常需要按照一定条件创建新的数据列,然后进行进一步分析.

#### 1.直接赋值
#### 2.df.apply
#### 3.df.assign方法
#### 4.按条件选择分组分别赋值

In [2]:
import pandas as pd

## 0.读取csv数据到dataframe

In [3]:
fpath = "beijing_last_12_weather.csv"
df = pd.read_csv(fpath)

In [4]:
df. head()

,日期,最高温,最低温,天气,风力风向
0,2024-12-01 星期日,15℃,-1℃,多云,西北风 2级
1,2024-12-02 星期一,4℃,-4℃,晴,西北风 3级
2,2024-12-03 星期二,9℃,-3℃,晴~多云,西北风 2级
3,2024-12-04 星期三,8℃,-2℃,晴,西北风 2级
4,2024-12-05 星期四,9℃,-2℃,晴~阴,西北风 3级


## 1.直接赋值的方法


#### 实例清理温度列,变成数字类型

In [5]:
df.loc[:,'最高温'] = df["最高温"].str.replace("℃", "").astype('int32')
df.loc[:,'最低温'] = df["最低温"].str.replace("℃", "").astype('int32')

In [6]:
df.head()

,日期,最高温,最低温,天气,风力风向
0,2024-12-01 星期日,15,-1,多云,西北风 2级
1,2024-12-02 星期一,4,-4,晴,西北风 3级
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级
3,2024-12-04 星期三,8,-2,晴,西北风 2级
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级


#### 实例:计算温度差(新增数据列)

In [8]:
# 意思是loc方法中的列名存在则执行修改，不存在则执行新增
# 注意:df["最高温"]是一个Series,后面的差的结果也是一个Serise
df.loc [:,"wencha"] = df["最高温"] - df["最低温"]

In [9]:
df.head()

,日期,最高温,最低温,天气,风力风向,wencha
0,2024-12-01 星期日,15,-1,多云,西北风 2级,16
1,2024-12-02 星期一,4,-4,晴,西北风 3级,8
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级,12
3,2024-12-04 星期三,8,-2,晴,西北风 2级,10
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级,11


## df.apply方法

### Apply afunction along an axis of the DataFrame,
### Objects passed to the function are Series objects whose index is either the DataFrame's index
### (axis=0) orthe DataFrame's columns (axis=1).

#### 实例:添加一列温度类型

####  1.如果最高温度大于33度就是高温
####  2.低于10度就是低温
####  3.否则是常温


In [12]:
def get_wendu_type(x):
    if x["最高温"]>33:
        return '高温'
    if x["最低温"]<-10:
        return '低温'
    return '常温'
# 注意要设置axis=1,这是series的index是columns,0时是行?
df.loc[:,'wendu_type'] = df.apply(get_wendu_type,axis = 1)

In [13]:
#查看温度,,,,类型计数.value_counts()
df['wendu_type'].value_counts()

wendu_type
常温    94
高温    17
低温     1
Name: count, dtype: int64

## 3. df.assign方法

### Assign new columns to a DataFrame.
### Returns a new object with all original columns in addition to new ones

### 实例: 将温度从摄氏度变为华氏度

In [15]:
df.assign(
    huamax = lambda x : x["最高温"]*9/5+32,
    huamin = df["最低温"]*9/5+32
)

,日期,最高温,最低温,天气,风力风向,wencha,wendu_type,huamax,huamin
0,2024-12-01 星期日,15,-1,多云,西北风 2级,16,常温,59.0,30.2
1,2024-12-02 星期一,4,-4,晴,西北风 3级,8,常温,39.2,24.8
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级,12,常温,48.2,26.6
3,2024-12-04 星期三,8,-2,晴,西北风 2级,10,常温,46.4,28.4
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级,11,常温,48.2,28.4
...,...,...,...,...,...,...,...,...,...
107,2025-10-08 星期三,18,10,中雨,南风 3级,8,常温,64.4,50.0
108,2025-10-09 星期四,14,10,中雨,东风 2级,4,常温,57.2,50.0
109,2025-10-10 星期五,15,11,中雨,南风 3级,4,常温,59.0,51.8
110,2025-11-01 星期六,27,14,晴,东北风 1级,13,常温,80.6,57.2


## 4.按条件选择分组分别赋值
#### 按照条件先选择符合条件的数据,然后对这部分数据赋值新列
#### 实例:高低温差大于10度,则认为温差大


In [17]:
# 先建空列(这是第一种创建新列的方法)
df["wencha_type"] = ''
df.loc[df["最高温"]-df["最低温"]>10,"wencha_type"] = "温差大"
df.loc[df["最高温"]-df["最低温"]<=10,"wencha_type"] = "温差正常"
df["wencha_type"].value_counts()

wencha_type
温差正常    82
温差大     30
Name: count, dtype: int64